# Watson OpenScale Fairness Metrics and Transformers

# 1. Introduction <a name="introduction"></a>
The notebook will train a German Credit Risk model, compute Fairness Metrics **Statistical Parity Difference** and **Smoothed Empirical Differential** on the model prediction and then show how **Fair Score Transformer** can be used to transform the model output for fair prediction.<br/>

This document includes below sections, you will need `edit` and `restart` notebook kernel in **Setup** section.

- [1.Introduction](#introduction)
- [2.Setup](#setup)
- [3.Model building and evaluation](#model)
- [4.OpenScale configuration](#openscale)
- [5.Compute Statistical Parity Difference with Original Scores](#spd)
- [6.Compute Smoothed Empirical Differential](#sed)
- [7.Fair Score Transformer](#fst)
- [8.Compute Statistical Parity Difference with Transformed Scores](#spd2)

**Note:** This notebook should be run using with **Python 3.10.x** runtime. It requires service credentials for the following services:
  * Watson OpenScale <br/>

## 2. Setup <a name="setup"></a>

### 2.1 Package installation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1
!pip install --upgrade pyspark==3.3.1 | tail -n 1

#### Action: restart the kernel!

### 2.2 Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [ ]:
WOS_CREDENTIALS = {
    "url": "<cluster-url>",
    "username": "<username>",
    "password": "<password>",
    "instance_id": "<openscale instance id>"
}

### 2.3 Run the notebook

&ensp;&ensp;&ensp;At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

## 3. Model building and evaluation <a name="model"></a>
&ensp;&ensp;&ensp;In this section you will learn how to train sklearn model, run prediction and evaluate its output. 

### 3.1 Load the training data from github

In [ ]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv

In [ ]:
import pandas as pd
data_df = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
data_df

### 3.2 Prepare data

In [ ]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, brier_score_loss
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
def convert(x):
    if x == "Risk":
        return 0
    else:
        return 1

In [ ]:
data_df["Risk"] = data_df["Risk"].apply(lambda x: convert(x))
data_df

### 3.3 Splitting the data into train and test

In [ ]:
train_data, test_data = train_test_split(data_df, test_size=0.2)

### 3.4 Create a model
&ensp;&ensp;&ensp;Preparing the pipeline. In this step you will encode target column labels into numeric values. 

In [ ]:
features_idx = np.s_[0:-1]
#all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In [ ]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log', penalty='l2', max_iter=1000, tol=1e-5)
pipeline_linear = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

&ensp;&ensp;&ensp;Train a model

In [ ]:
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)

### 3.5 Evaluate the model
&ensp;&ensp;&ensp;Run the model to get predict class labels and probability estimates for test data

In [ ]:
y_preds = risk_model.predict(test_data.drop('Risk', axis=1))
y_probs = risk_model.predict_proba(test_data.drop('Risk', axis=1))[:,1]

&ensp;&ensp;&ensp;Compute accuracy and loss with model output

In [ ]:
from sklearn.metrics import accuracy_score, brier_score_loss

In [ ]:
lr_acc = accuracy_score(test_data['Risk'], y_preds)
print(lr_acc)

In [ ]:
lr_brier = brier_score_loss(test_data['Risk'], y_probs)
print(lr_brier)

## 4. OpenScale configuration <a name="openscale"></a>
&ensp;&ensp;&ensp;The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient as OpenScaleAPIClient
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

authenticator = CloudPakForDataAuthenticator(
    url=WOS_CREDENTIALS["url"],
    username=WOS_CREDENTIALS["username"],
    password=WOS_CREDENTIALS["password"],
    disable_ssl_verification=True
)

client = OpenScaleAPIClient(
    service_url=WOS_CREDENTIALS['url'],
    service_instance_id=WOS_CREDENTIALS["instance_id"],
    authenticator=authenticator
)

client.version

## 5. Compute Statistical Parity Difference with Original Scores <a name="spd"></a>

**Statistical Parity Difference** is a fairness metric that can be used to describe the fairness for the model predictions.
It is the difference between the ratio of favourable outcomes in unprivileged and privileged groups. It can
be computed from either the input dataset or the dataset output from a classifier (predicted dataset). A value
of 0 implies both groups have equal benefit, a value less than 0 implies higher benefit for the privileged group, and a value greater than 0 implies higher benefit for the unprivileged group.<br>
$$𝑃(𝑌=1|𝐷=unprivileged)−𝑃(𝑌=1|𝐷=privileged)$$

Take the German credit risk datasets as example, if user set
+ privileged group as Sex="male" 
+ unprivileged group as Sex="female"

and set
+ favourable label as Risk="No Risk"
+ unfavourable label as Risk="Risk"

then, the SPD result 
+ spd > 0 means the unpriviliage group Sex="female" has higher rate to be marked as favourable label "No Risk" than priviliage group Sex="male".
+ spd = 0 means the unpriviliage group Sex="female" has same rate to be marked as favourable label "No Risk" with priviliage group Sex="male".
+ spd < 0 means the unpriviliage group Sex="female" has lower rate to be marked as favourable label "No Risk" than priviliage group Sex="male".

&ensp;&ensp;&ensp;Add a new column `pred` with value of the model predictions

In [ ]:
test_data["pred"] = y_preds
test_data

### 5.1 Prepare input to compute Statistical Parity Difference

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.bindAddress", "127.0.0.1").getOrCreate()

In [ ]:
sparkDF=spark.createDataFrame(test_data)

Setup configuration to compute *Statistical Parity Difference*,<br/>

Configure label and problem type in the overall section.
- **problem_type(str)**: `binary` and `multi-classification` is supported.
- **label_column(str)**: Column name of label in the data frame

Inside `fairness` as below, there are three sections which is required to configure.
- **metrics_configuration(dict)**: Configure *Statistical Parity Difference* as one of the metrics with name `FairnessMetricType.SPD.value`, and it requires a `features` property to describe which features the metric will be computed upon. *Statistical Parity Difference* is supported to run with individual features (eg. `[["a"],["b"]]`), but not suppored to run with intersectional features (eg. `[["a", "b"]]`).

- **protected_attributes(list)**: Describe privileged group defintion for features upon which this metric will be computed. Configure each feature with below information:
  - feature(str): Name of the feature, which should be same as configured in `features` of `metrics_configuration` section.
  - reference_group(list): List of feature values which make a sample privileged. 

- **favourable_label(list)**: A list of favourable labels or outcomes of the model.


**Note** that `label_column` used here is the new added `pred` column.<br/>

In [ ]:
from ibm_metrics_plugin.common.utils.constants import FairnessMetricType

spd_config = {}
spd_config['configuration'] = {
            "problem_type": "binary",
            "label_column": "pred",
            "fairness": {
                         "metrics_configuration": {
                                    FairnessMetricType.SPD.value: {
                                        "features": [["Sex"]]
                                    }
                        },
                        "protected_attributes": [
                            {
                                "feature": "Sex",
                                "reference_group": ["male"]
                            }
                        ],
                        "favourable_label": ["1"]          
            }  
        }

### 5.2 Compute Statistical Parity Difference

In [ ]:
metrics = client.ai_metrics.compute_metrics(spark=spark, configuration=spd_config, data_frame=sparkDF)

In [ ]:
metrics

## 6. Compute Smoothed Empirical Differential Fairness <a name="sed"></a>

**Smoothed Empirical Differential(SED)** is a fairness metric that can be used to describe the fairness for the model predictions. It is used to quantify the differential in the probability of favorable/unfavorable outcomes between intersecting groups divided by features. All intersecting groups are equal, there is no unprivileged or privileged groups. 

SED value is the minimum ratio of Dirichlet smoothed probability of favorable and unfavorable outcomes between different intersecting groups in the dataset. Its value is between 0 and 1, excluding 0 and 1. The bigger, the better.

Take the German credit risk datasets as example, assume:

+ the favorable outcomes of label column is "No Risk",
+ the unfavorable outcomes of label column is "Risk".

if user divide dataset by *feature "Sex"*，there will be two intersecting groups:
+ intersecting group Sex="male" 
+ intersecting group Sex="female"

and assume:

+ the Dirichlet smoothed probability of favorable outcomes "No Risk" in intersecting group "Sex"="male" is 0.2
+ the Dirichlet smoothed probability of unfavorable outcomes "Risk" in intersecting group "Sex"="male" is 0.8
+ the Dirichlet smoothed probability of favorable outcomes "No Risk" in intersecting group "Sex"="female" is 0.4
+ the Dirichlet smoothed probability of unfavorable outcomes "Risk" in intersecting group "Sex"="female" is 0.6

then, calculate the label differential between intersecting groups (*Note that it always chooses the smaller one as the numerator or the bigger one as the denominator*): 

+ the favorable outcomes' differential between intersecting group "Sex"="male" and "Sex"="female" will be 0.2/0.4=0.5
+ the unfavorable outcomes' differential between intersecting group "Sex"="male" and "Sex"="female" will be 0.6/0.8=0.75

then, calculate the differential between intersecting groups:
+ the differential between intersecting group "Sex"="male" and "Sex"="female" will be min(0.5, 0.75)=0.5

Since there are only two intersecting groups, so,

+ the final differentials of dataset will be 0.5.

*References: James R. Foulds, Rashidul Islam, Kamrun Naher Keya, Shimei Pan, "An Intersectional Definition of Fairness", Department of Information Systems, University of Maryland, Baltimore County, USA*


### 6.1 Smoothed Empirical Differential Configuration

Configure label and problem type in the overall section.
- **problem_type(str)**: `binary` and `multi-classification` is supported.
- **label_column(str)**: Column name of label in the data frame.

Inside `fairness` as below, there are three sections which is required to configure.
- **metrics_configuration(dict)**: Configure *Smoothed Empirical Differential* as one of the metrics with name `FairnessMetricType.SED.value`, and it requires a `features` property to describes which features the metric will be computed upon. *Smoothed Empirical Differential* is supported to run with individual features (eg. `[["a"],["b"]]`) and with intersectional features (eg. `[["a", "b"]]`).

- **protected_attributes(list)**: Describe protected features upon which this metric will be computed. Configure each feature with such information:
  - feature(str): Name of the feature, which should be same as configured in `features` of `metrics_configuration` section.

- **favourable_label(list)**: A list of favourable labels or outcomes of the model.

In [ ]:
sed_config = {}
sed_config['configuration'] = {
            "problem_type": "binary",
            "label_column": "pred",
            "fairness": {
                         "metrics_configuration": {
                                    FairnessMetricType.SED.value: {
                                        "features": [["Sex"]],
                                    }
                        },
                        "protected_attributes": [
                            {
                                "feature": "Sex"
                            }
                        ],
                        "favourable_label": ["1"]
            }  
        }

### 6.2 Compute Smoothed Empirical Differential

In [ ]:
metrics = client.ai_metrics.compute_metrics(spark=spark, configuration=sed_config, data_frame=sparkDF)

In [ ]:
metrics

## 7. Fair Score Transformer <a name="fst"></a>

**Fair Score Transformer** can be used as post-processing technique that transforms probability estimates ( or scores) of `probabilistic binary classication` model with respect to fairness goals like statistical parity or equalized odds. To use **Fair Score Transformer** in OpenScale, you need first train a **Fair Score Transformer** and then use it to transform scores.

*References: D. Wei, K. Ramamurthy, and F. Calmon, "Optimized Score Transformation for Fair Classification", International Conference on Artificial Intelligence and Statistics, 2020.* 

### 7.1 Train Fair Score Transformer

To train Fair Score Transformer, at least two columns is required in the dataframe, one is the probability estimates from the trained classification model and another is the corresponding protected attributes.

**Note**
The `label` column is not required to train the transformer but required to compute accuray with the trained transformer later.

In [ ]:
#probability estimates
r = y_probs.tolist()
#protected attributes
A = test_data["Sex"].tolist()
#label values
y = test_data["Risk"].tolist()

In [ ]:
r_name = "r"
A_name = "A"
y_name = "y"
data = pd.DataFrame({
    r_name: r,
    A_name: A,
    y_name: y
})
data

In [ ]:
sparkDF=spark.createDataFrame(data)
sparkDF.show()

### 7.2 Fair Score Transformer Configuration

Setup configuration to fit **Fair Score Transformer**. Inside `metrics_configuration` as below, specify the name of the transformer with `FairnessMetricType.FST.value`. To configure it, you need to provide `params` and `features` information as below. This notebook will transform scores with respect to the **Statistical Parity Difference** fairness goal (set `criteria` as `MSP`).

- **params**: Parameters of Fair Score Transformer
  - epsilon (float): Bound on mean statistical parity or mean equalized odds.
  - criteria (str): Optimize for mean statistical parity ("MSP") or mean equalized odds ("MEO").
  - Aprobabilistic (bool): Indicator of whether actual protected attribute values (False) or probabilistic estimates (True) are provided. Default False.
  - iterMax (float): Maximum number of ADMM iterations. Default 1e3.
- **features**: Columns definition in the dataframe
  - probabilities: Column name of probability estimates.
  - protected: Column name of protected attributes.


In [ ]:
columns = {"probabilities": r_name, "protected": A_name}
configuration = dict()
configuration["configuration"] = {
    "fairness": {
        "metrics_configuration": {
            FairnessMetricType.FST.value: {
                "params": {
                    "epsilon": 0.01,
                    "criteria": "MSP",
                    "Aprobabilistic": False,
                    "iterMax": 1e3
                },
                "features": columns
            }
        }     
    }
}

&ensp;&ensp;&ensp;Fit fair score transformer

In [ ]:
fst = client.ai_metrics.fit_transformer(spark=spark, configuration=configuration, data_frame=sparkDF)

### 7.3 Transform scores with Fair Score Transformer

&ensp;&ensp;&ensp;Trained transformer can be used to compute new probability estimates and it requires the exactly same columns as fitting phase.<br/> 

&ensp;&ensp;&ensp;**Note:** No matter what column name is used for the existing probability estimates, the new probability estimates column will be named as **r_transformed**.

In [ ]:
probs_df = fst.predict_proba(spark, sparkDF, columns, keep_cols=[y_name])
probs_df.show()

&ensp;&ensp;&ensp;Trained transformer can also be used to compute new class labels based on transformed probability estimates, and it requires the exactly same columns as fitting phase. 

&ensp;&ensp;&ensp;**Note:** No matter what column name is used for the `label` column, the new class labels column will be named as **r_transformed_thresh**.

In [ ]:
preds_df = fst.predict(spark, sparkDF, columns, keep_cols=[y_name])
preds_df.show()

### 7.4 Evaluate with transformed scores

&ensp;&ensp;&ensp;To compute accuray based on transformed probability estimates with the trained transformer, you need to specify the `label` column name.

In [ ]:
score_columns = columns.copy()
score_columns["label"] = "y"

In [ ]:
acc = fst.score(spark, sparkDF, score_columns)
acc

&ensp;&ensp;&ensp;Or you can get the predict class labels from transformer and compute accuracy directly.

In [ ]:
fst_preds = preds_df.select("r_transformed_thresh").toPandas().values.ravel()

In [ ]:
lr_acc = accuracy_score(test_data['Risk'], y_preds)
fst_lr_acc = accuracy_score(test_data['Risk'], fst_preds)
print("Original model accuray: {}".format(lr_acc))
print("Accuracy with transformed predicts: {}".format(fst_lr_acc))

&ensp;&ensp;&ensp;You can compute loss with transformed probability estimates too.

In [ ]:
fst_probs = probs_df.select("r_transformed").toPandas().values.ravel()

In [ ]:
fst_lr_brier = brier_score_loss(test_data['Risk'], fst_probs)
lr_brier = brier_score_loss(test_data['Risk'], y_probs)
print("Original model loss: {}".format(lr_acc))
print("Loss with transformed scores: {}".format(fst_lr_acc))

## 8. Compute Statistical Parity Difference  with Transformed Scores <a name="spd2"></a>

&ensp;&ensp;&ensp;Compute **Statistical Parity Difference** based on the transformed class labels. <br/>
&ensp;&ensp;&ensp;Add a new column `pred_transformed` with value of the transformed class labels.

In [ ]:
test_data["pred_transformed"] = fst_preds
test_data

&ensp;&ensp;&ensp;Prepare input to compuate **Statistical Parity Difference**. <br/>
&ensp;&ensp;&ensp;All will be the same as before except the `label_column` will be swithed to use the transformed class labels column.

In [ ]:
spd_config['configuration']['label_column'] = "pred_transformed"

In [ ]:
sparkDF=spark.createDataFrame(test_data)

&ensp;&ensp;&ensp;Compute **Statistical Parity Difference** again and it should be improved compared with data before transformed.

In [ ]:
metrics = client.ai_metrics.compute_metrics(spark=spark, configuration=spd_config, data_frame=sparkDF)

In [ ]:
metrics